In [56]:
description = "Imbued on every page with Frank McCourt's astounding humor and compassion. This is a glorious book that bears all the marks of a classic.When I look back on my childhood I wonder how I managed to survive at all. It was, of course, a miserable childhood: the happy childhood is hardly worth your while. Worse than the ordinary miserable childhood is the miserable Irish childhood, and worse yet is the miserable Irish Catholic childhood.So begins the Pulitzer Prize winning memoir of Frank McCourt, born in Depression-era Brooklyn to recent Irish immigrants and raised in the slums of Limerick, Ireland. Frank's mother, Angela, has no money to feed the children since Frank's father, Malachy, rarely works, and when he does he drinks his wages. Yet Malachy-- exasperating, irresponsible and beguiling-- does nurture in Frank an appetite for the one thing he can provide: a story. Frank lives for his father's tales of Cuchulain, who saved Ireland, and of the Angel on the Seventh Step, who brings his mother babies. Perhaps it is story that accounts for Frank's survival. Wearing rags for diapers, begging a pig's head for Christmas dinner and gathering coal from the roadside to light a fire, Frank endures poverty, near-starvation and the casual cruelty of relatives and neighbors--yet lives to tell his tale with eloquence, exuberance and remarkable forgiveness. Angela's Ashes, imbued on every page with Frank McCourt's astounding humor and compassion, is a glorious book that bears all the marks of a classic."

In [57]:
# Example usage
occupied = 0
location = [6.903086, 79.878333]
print(type(location))

<class 'list'>


In [58]:
import pandas as pd
import geopy.distance
from keras.models import load_model
import tokanizer_and_cleaner
import numpy as np

# Step 1: Load the user_book.csv dataset
df = pd.read_csv('E:\\RESEARCH\\rec\\bookgramb\\data\\books_with_users.csv')

# Step 2: Load the saved LSTM model
model = load_model('E:\\RESEARCH\\rec\\bookgramb\\models\\best_model.h5')

# Step 3: Preprocess the text data in the description column of the df dataframe
max_desc_length=200

labels = ['fiction', 'nonfiction']
cleaned_text = tokanizer_and_cleaner.clean_text(description)
tokenized_text = tokanizer_and_cleaner.tokenizer(cleaned_text, tokanizer_and_cleaner.vocab_dict, max_desc_length)
input_data = np.reshape(tokenized_text, (1, max_desc_length))
# Step 4: Use the loaded LSTM model to predict the genre of books
output = model.predict(input_data, batch_size=1)
score = (output > 0.5) * 1
pred = score.item()


# Step 5: Compare the predicted genre with the actual genre in the label column of the df dataframe
df['predicted_genre'] = labels[pred]

# Step 6: Filter the top 10 books with the predicted genre
top10_books = df[df['predicted_genre'] == df['label']]

# Step 7: Print the top 10 books with the predicted genre
print(top10_books)


1/1 [==============================] - 1s 1s/step
       book_id               book_authors   
75          75              Frank McCourt  \
112        112  Mitch Albom|Saulius Dagys   
126        126              Truman Capote   
156        156          Elizabeth Gilbert   
201        201            Thomas Keneally   
...        ...                        ...   
45996    45996          Natascha Kampusch   
46007    46007              Avi Steinberg   
46008    46008              Howard Megdal   
46009    46009              Howard Megdal   
46010    46010     Mimi Baird|Eve Claxton   

                                               book_desc   
75     Imbued on every page with Frank McCourt's asto...  \
112    Maybe it was a grandparent, or a teacher or a ...   
126    On November 15, 1959, in the small town of Hol...   
156    A celebrated writer's irresistible, candid, an...   
201    In the shadow of Auschwitz, a flamboyant Germa...   
...                                              

In [59]:
# Define the rule-based recommendation function
def rule_based_recommendation(top10_books, occupied):
    if occupied == 1:
        # If the user is currently occupying a location, recommend books with high ratings
        recommendations = top10_books[top10_books['book_rating'] >= 4.0].sort_values(by=['book_rating'], ascending=False)
    else:
        # If the user is not currently occupying a location, recommend popular books
        recommendations = top10_books.sort_values(by=['book_rating_count', 'book_review_count'], ascending=False)

    # Return the top 10 recommendations
    return recommendations.head(10)

# Define the geographical recommendation function
def geographical_recommendation(top10_books, location):
    # Extract the latitude and longitude values from the location string
    top10_books['latitude'] = top10_books['location'].apply(lambda x: float(x.strip('[]').split(',')[0]))
    top10_books['longitude'] = top10_books['location'].apply(lambda x: float(x.strip('[]').split(',')[1]))

    # Compute the distance between each user and the target location
    top10_books['distance'] = top10_books.apply(lambda row: geopy.distance.distance((row['latitude'], row['longitude']), location).km, axis=1)
    # Sort the books by distance
    recommendations = top10_books.sort_values(by=['distance'])
    # print(recommendations)
    # Return the top 10 recommendations
    return recommendations.head(10)

# Define the hybrid recommendation function
def hybrid_recommendation(top10_books, occupied, location):
    # Get the recommendations from the rule-based and geographical recommendation systems
    rule_based_recommendations = rule_based_recommendation(top10_books, occupied)
    geographical_recommendations = geographical_recommendation(rule_based_recommendations, location)

    # Combine the two sets of recommendations and remove duplicates
    recommendations = pd.concat([rule_based_recommendations, geographical_recommendations])
    recommendations = recommendations.drop_duplicates(subset='book_id')

    # Return the top 10 recommendations
    return recommendations.head(10)

recommendations = hybrid_recommendation(df, occupied, location)
print(recommendations[['book_title', 'book_rating', 'distance']])

                                  book_title  book_rating   distance
46     Harry Potter and the Sorcerer's Stone         4.46  20.628801
19608                       The Hunger Games         4.33  57.433685
20016                       The Hunger Games         4.33  10.622522
13904                       The Hunger Games         4.33  24.272735
0                           The Hunger Games         4.33  12.508600
23064                               Twilight         3.58  96.190179
3                                   Twilight         3.58  87.755512
29639                  To Kill a Mockingbird         4.27  77.777584
27061                  To Kill a Mockingbird         4.27  79.783722
22966                  To Kill a Mockingbird         4.27  42.778476
